# Loading data

In [1]:
import json

with open('clinais.train.json') as f:
    data = json.load(f)

from tqdm import tqdm

finalresult = []
for key in tqdm(data['annotated_entries'].keys()):
    ident = data['annotated_entries'][key]['note_id']
    res = []
    tags = []
    gold = data['annotated_entries'][key]['boundary_annotation']['gold']
    currentboundary = ''
    for g in gold:
        res.append(g['span'])
        if(g['boundary'] is None):
            tags.append('I-'+currentboundary)
        else:
            currentboundary = g['boundary']
            tags.append('B-'+currentboundary)
    finalresult.append([ident,res,tags])

# finalresult    

import numpy as np
import itertools
tags = [x[2] for x in finalresult]
tags = np.unique(list(itertools.chain(*tags)))
id2label = {}
label2id = {}
for i,tag in enumerate(tags):
    id2label[i] = tag
    label2id[tag] = i

100%|██████████| 781/781 [00:00<00:00, 8481.68it/s]


In [2]:
from datasets import load_from_disk

dataset = load_from_disk('augmented_dataset')

/home/joheras/.local/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


# Processing dataset

In [3]:
from transformers import AutoTokenizer

In [4]:
modelCheckpoint = "joheras/bsc-bio-ehr-es-finetuned-clinais"
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)

In [5]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [6]:
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/joheras/CLINAIS/augmented_dataset/train/cache-c9fa691dc6276bf8.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2850
    })
    val: Dataset({
        features: ['tokens', 'tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 127
    })
})

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:
import evaluate

seqeval = evaluate.load("seqeval")

In [10]:
import numpy as np



def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [11]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    modelCheckpoint, num_labels=len(id2label), id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at joheras/bsc-bio-ehr-es-finetuned-clinais were not used when initializing RobertaForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at joheras/bsc-bio-ehr-es-finetuned-clinais and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on

In [12]:
training_args = TrainingArguments(
    output_dir="bsc-bio-ehr-es-finetuned-clinais-augmented1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/joheras/CLINAIS/bsc-bio-ehr-es-finetuned-clinais-augmented1 is already a clone of https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented1. Make sure you pull the latest changes with `repo.git_pull()`.
The following columns in the training set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
/home/joheras/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2850
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 3

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.624553,0.275920,0.348469,0.307979,0.828556
2,No log,0.589010,0.351107,0.451954,0.395199,0.840231
3,No log,0.597340,0.368960,0.532207,0.435798,0.853912
4,No log,0.667581,0.369283,0.571278,0.448590,0.855354
5,No log,0.717313,0.422680,0.606125,0.498048,0.854421
6,0.357900,0.785386,0.443760,0.608237,0.513140,0.855156
7,0.357900,0.843738,0.452978,0.610348,0.520018,0.852244
8,0.357900,0.871610,0.434913,0.610348,0.507909,0.851340
9,0.357900,0.886821,0.449961,0.602957,0.515343,0.854365
10,0.357900,0.891683,0.467984,0.625132,0.535262,0.857417


The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: tokens, tags. If tokens, tags are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 127
  Batch size = 32
/grupoa/config/miniconda3/envs/fastai/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to bsc-bio-ehr-es-finetuned-clinais-augmented1/checkpoint-90
Configuration saved in bsc-bio-ehr-es-finetuned-clinais-augmented1/checkpoint-90/config.json
Model weights saved in bsc-bio-ehr-es-finetuned-clinais-augmented1/checkpoint-90/pytorch_model.bin
tokenizer config file saved in bsc-bio-ehr-es-finetuned

TrainOutput(global_step=9000, training_loss=0.022554755688541466, metrics={'train_runtime': 5716.4146, 'train_samples_per_second': 49.856, 'train_steps_per_second': 1.574, 'total_flos': 7.447072679062989e+16, 'train_loss': 0.022554755688541466, 'epoch': 100.0})

In [13]:
trainer.push_to_hub()

Saving model checkpoint to bsc-bio-ehr-es-finetuned-clinais-augmented1
Configuration saved in bsc-bio-ehr-es-finetuned-clinais-augmented1/config.json
Model weights saved in bsc-bio-ehr-es-finetuned-clinais-augmented1/pytorch_model.bin
tokenizer config file saved in bsc-bio-ehr-es-finetuned-clinais-augmented1/tokenizer_config.json
Special tokens file saved in bsc-bio-ehr-es-finetuned-clinais-augmented1/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/473M [00:00<?, ?B/s]

Upload file runs/Mar20_17-50-01_minion/events.out.tfevents.1679331005.minion.382827.0:  59%|#####9    | 32.0k/…

To https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented1
   1705bbe..f750dc9  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.5320069204152249}, {'name': 'Recall', 'type': 'recall', 'value': 0.6494192185850053}, {'name': 'F1', 'type': 'f1', 'value': 0.5848787446504994}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.8582654907281773}]}
To https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented1
   f750dc9..7fefaee  main -> main



'https://huggingface.co/joheras/bsc-bio-ehr-es-finetuned-clinais-augmented1/commit/f750dc974d81c61d0f20e75cc8aabdff4ac4a265'

In [14]:
!rm -rf bsc-bio-ehr-es-finetuned-clinais-augmented1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
